In [1]:
# CELL 0 — (optional) show GPU & free memory
!nvidia-smi -L
!nvidia-smi

GPU 0: Tesla T4 (UUID: GPU-b389e076-f280-a708-c183-fde045298406)
GPU 1: Tesla T4 (UUID: GPU-bb3f6682-368e-2d69-73e3-6ad0bc9c096e)
Mon Oct  6 11:42:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8             11W /   70W |       1MiB /  15360MiB |      0%      Default |
|         

In [2]:
# CELL 1 — install required packages (run once)
# use -q to reduce noise; versions can be changed if needed
!pip install -q transformers accelerate bitsandbytes datasets peft evaluate safetensors


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 29.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# CELL 2 — imports + reproducibility
import os, re, random, math, json, gc
from pathlib import Path
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)


2025-10-06 11:45:48.934353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759751149.286259      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759751149.393695      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Device: cuda


In [5]:
# CELL 3 — load training_set_rel3.tsv
import pandas as pd, os

DATA_PATH = "/kaggle/input/training-set/training_set_rel3.tsv"  # adjust if path differs
df = pd.read_csv(DATA_PATH, sep="\t", encoding="latin-1")

print("Columns:", df.columns.tolist())
print("Total essays:", len(df))

# keep only relevant columns
cols_needed = ["essay_id", "essay_set", "essay", "domain1_score"]
df = df[cols_needed]
df.head()


Columns: ['essay_id', 'essay_set', 'essay', 'rater1_domain1', 'rater2_domain1', 'rater3_domain1', 'domain1_score', 'rater1_domain2', 'rater2_domain2', 'domain2_score', 'rater1_trait1', 'rater1_trait2', 'rater1_trait3', 'rater1_trait4', 'rater1_trait5', 'rater1_trait6', 'rater2_trait1', 'rater2_trait2', 'rater2_trait3', 'rater2_trait4', 'rater2_trait5', 'rater2_trait6', 'rater3_trait1', 'rater3_trait2', 'rater3_trait3', 'rater3_trait4', 'rater3_trait5', 'rater3_trait6']
Total essays: 12976


,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [6]:
# CELL 4 — select only Essay Set 1 and basic cleanup
ESSAY_TOPIC = (
    "More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about far away places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends. Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you."
)

df = df[df["essay_set"] == 1].copy()
df = df.dropna(subset=["essay", "domain1_score"]).reset_index(drop=True)

df["essay"] = df["essay"].astype(str).str.strip()
df["domain1_score"] = df["domain1_score"].astype(int)

print("Essay Set 1 samples:", len(df))
df.head()


Essay Set 1 samples: 1783


,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


In [10]:
train_df, temp_df = train_test_split(df, test_size=0.30, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [11]:
print(len(train_df))
print(len(val_df))
print(len(test_df))

1248
267
268


In [12]:
# CELL 6 — convert to HuggingFace Datasets
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)
dataset = DatasetDict({"train": train_ds, "validation": val_ds, "test": test_ds})

In [13]:
# CELL 7 — prompt + label creation (uses essay topic as context)

SYSTEM_PROMPT = (
    "You are an experienced English teacher grading a student's essay.\n\n"
    "Read the essay carefully and assign a score \n\n"
    "Essay Prompt:\n"
    f"{ESSAY_TOPIC}\n\n"
    "Instructions:\n"
    "- The score must be a single integer between 2 and 12 (inclusive).\n"
    "- Do not provide any explanation or reasoning.\n"
    "- Output only the score number.\n\n"
    "Now, read the following essay and provide the score.\n"
)

PROMPT_SUFFIX = "\n\nEssay:\n"
SCORE_SUFFIX = "\n\nScore: "


def make_full_input(example):
    essay = example["essay"].strip()
    score = str(int(example["domain1_score"]))
    prompt = SYSTEM_PROMPT + PROMPT_SUFFIX + essay + SCORE_SUFFIX
    full = prompt + score
    return {"prompt": prompt, "full": full, "score_str": score}

dataset = dataset.map(lambda x: make_full_input(x), remove_columns=dataset["train"].column_names)
dataset = dataset.rename_columns({"prompt": "prompt_text", "full": "text", "score_str": "score_text"})
dataset


Map:   0%|          | 0/1248 [00:00<?, ? examples/s]

Map:   0%|          | 0/267 [00:00<?, ? examples/s]

Map:   0%|          | 0/268 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt_text', 'text', 'score_text'],
        num_rows: 1248
    })
    validation: Dataset({
        features: ['prompt_text', 'text', 'score_text'],
        num_rows: 267
    })
    test: Dataset({
        features: ['prompt_text', 'text', 'score_text'],
        num_rows: 268
    })
})

In [14]:
# CELL 8 — tokenizer + tokenization mapping
MODEL_NAME = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit"  # prefer a pre-quantized community HF repo if available; fallback below
# fallback: "Qwen/Qwen2.5-3B"
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True, use_fast=False)
except Exception as e:
    print("Primary model tokeniser failed; falling back to official name. Error:", e)
    MODEL_NAME = "Qwen/Qwen2.5-3B"
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True, use_fast=False)

# Ensure padding token exists:
if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({"pad_token":"<|pad|>"})

MAX_LENGTH = 1024  # keep moderate for VRAM; Qwen supports long contexts, but T4 is limited
print("Tokenizer loaded. Pad token id:", tokenizer.pad_token_id)

# tokenization function that produces input_ids and labels (masking prompt tokens)
def tokenize_and_mask(examples):
    outputs = {"input_ids": [], "attention_mask": [], "labels": []}
    for prompt, full in zip(examples["prompt_text"], examples["text"]):
        tokenized_full = tokenizer(full, truncation=True, max_length=MAX_LENGTH)
        tokenized_prompt = tokenizer(prompt, truncation=True, max_length=MAX_LENGTH)
        input_ids = tokenized_full["input_ids"]
        attention_mask = tokenized_full["attention_mask"]
        labels = input_ids.copy()
        # mask the prompt tokens
        prompt_len = len(tokenized_prompt["input_ids"])
        labels[:prompt_len] = [-100] * prompt_len
        outputs["input_ids"].append(input_ids)
        outputs["attention_mask"].append(attention_mask)
        outputs["labels"].append(labels)
    return outputs

# apply tokenization
tokenized = dataset.map(tokenize_and_mask, batched=True, remove_columns=dataset["train"].column_names)
# make sure data are in torch format at training time via collator
tokenized


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

Tokenizer loaded. Pad token id: 151654


Map:   0%|          | 0/1248 [00:00<?, ? examples/s]

Map:   0%|          | 0/267 [00:00<?, ? examples/s]

Map:   0%|          | 0/268 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1248
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 267
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 268
    })
})

In [15]:
# CELL 9 — small custom collator to pad labels with -100
import torch
def data_collator(batch):
    input_ids = [torch.tensor(x["input_ids"], dtype=torch.long) for x in batch]
    labels = [torch.tensor(x["labels"], dtype=torch.long) for x in batch]
    attention = [torch.tensor(x["attention_mask"], dtype=torch.long) for x in batch]
    input_ids_padded = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    labels_padded = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)
    attention_padded = torch.nn.utils.rnn.pad_sequence(attention, batch_first=True, padding_value=0)
    return {"input_ids": input_ids_padded, "attention_mask": attention_padded, "labels": labels_padded}


In [16]:
# CELL 10 — load the model in 4-bit with bitsandbytes config (QLoRA style)
import gc
gc.collect()
torch.cuda.empty_cache()

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",            # nf4 is common for QLoRA
    bnb_4bit_compute_dtype=torch.float16
)

print("Loading model (this can take a minute)...")
try:
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
                                                 quantization_config=bnb_config,
                                                 trust_remote_code=True,
                                                 device_map="auto")
except Exception as e:
    # fallback: try official repo
    print("Primary load failed, trying official Qwen name. Err:", e)
    MODEL_NAME = "Qwen/Qwen2.5-3B"
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME,
                                                 quantization_config=bnb_config,
                                                 trust_remote_code=True,
                                                 device_map="auto")
print("Model loaded.")


Loading model (this can take a minute)...


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

Model loaded.


In [17]:
# CELL 11 — prepare for k-bit + attach LoRA (PEFT)
model = prepare_model_for_kbit_training(model)  # necessary before get_peft_model with 4bit
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],   # common choice that targets attention projections; can add k_proj/o_proj if you like
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # sanity check


trainable params: 1,843,200 || all params: 3,087,781,888 || trainable%: 0.0597


In [21]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./qwen2.5-asap-p1-lora",
    per_device_train_batch_size=1,        # safe for T4
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,       # simulate batch of 16
    num_train_epochs=1,                   # complete within ~1 hour
    learning_rate=3e-5,                   # faster convergence since 1 epoch
    fp16=True,
    optim="paged_adamw_8bit",             # memory-efficient optimizer
    logging_steps=25,                     # log every 25 steps
    eval_strategy="steps",          # evaluate during training
    eval_steps=200,                       # run eval every 200 steps
    save_strategy="steps",                # save model periodically
    save_steps=200,                       # save every 200 steps (~every 10-15 mins)
    save_total_limit=None,                # keep ALL checkpoints
    load_best_model_at_end=True,          # loads lowest eval loss checkpoint
    ddp_find_unused_parameters=False,
    report_to="none",                     # disable wandb etc
    logging_dir="./logs",              
    # store logs
    warmup_ratio=0.1,                     # small warmup helps stability
    gradient_checkpointing=True,          # saves VRAM
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)


/tmp/ipykernel_36/123878270.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [22]:
# CELL 13 — train (this will run on the T4)
trainer.train()
# after training you can save the LoRA adapters only (small)
model.save_pretrained("./qwen2.5-asap-p1-lora-adapter")


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss


In [23]:
# CELL 14 — evaluation: loss on test set (uses labels prepared earlier)
metrics = trainer.evaluate(tokenized["test"])
print("Eval metrics (from Trainer):", metrics)
# metrics should contain eval_loss; if not, you can compute CE loss manually below


Eval metrics (from Trainer): {'eval_loss': nan, 'eval_runtime': 112.5344, 'eval_samples_per_second': 2.381, 'eval_steps_per_second': 2.381, 'epoch': 1.0}


In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel   # <-- THIS WAS MISSING

DEVICE = "cuda:0"

MODEL_PATH = "unsloth/Qwen2.5-3B-Instruct-bnb-4bit"                  # your base model path
LORA_PATH = "/kaggle/working/qwen2.5-asap-p1-lora-adapter"       # your fine-tuned LoRA folder

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(MODEL_PATH).to(DEVICE)

# Load fine-tuned LoRA weights on top
model = PeftModel.from_pretrained(base_model, LORA_PATH).to(DEVICE)

model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 2048, padding_idx=151654)
        (layers): ModuleList(
          (0-35): 36 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
          

In [34]:
# CELL 15 — Efficient test evaluation
import re
import torch

from transformers import logging
logging.set_verbosity_error()

def predict_single_score(essay_text):
    # Truncate essay to reduce token length
    prompt = SYSTEM_PROMPT + PROMPT_SUFFIX + essay_text.strip()[:2000] + SCORE_SUFFIX
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).to(DEVICE)

    with torch.no_grad():
        # Greedy generation
        generated = model.generate(
            **inputs,
            max_new_tokens=8,
            do_sample=False,               # greedy
            pad_token_id=tokenizer.pad_token_id
        )

    gen_text = tokenizer.decode(
        generated[0][inputs["input_ids"].shape[1]:], 
        skip_special_tokens=True
    )
    
    m = re.search(r"\b(\d{1,2})\b", gen_text)
    if m:
        return int(m.group(1)), gen_text.strip()
    else:
        return None, gen_text.strip()


# Process essays in small batches to avoid GPU bottleneck
batch_size = 8  # adjust based on GPU memory
gold = test_df['domain1_score'].tolist()
essays = test_df['essay'].tolist()

preds = []
raw_outs = []

for start in range(0, len(essays), batch_size):
    batch_essays = essays[start:start+batch_size]
    for e in batch_essays:
        p, raw = predict_single_score(e)
        preds.append(p)
        raw_outs.append(raw)
    print(f"Processed {len(preds)}/{len(essays)} essays")


# Compute exact-match accuracy
correct = sum(1 for a, b in zip(preds, gold) if a is not None and a == b)
total = len(gold)
accuracy = correct / total
print(f"Test exact-match accuracy: {accuracy:.4f} ({correct}/{total})")

# Show some examples where model prediction was wrong
for i, (g, p, raw) in enumerate(zip(gold, preds, raw_outs)):
    if i < 10 and p != g:
        print("GOLD", g, "PRED", p, "RAWOUT", repr(raw))


Processed 8/268 essays
Processed 16/268 essays
Processed 24/268 essays
Processed 32/268 essays
Processed 40/268 essays
Processed 48/268 essays
Processed 56/268 essays
Processed 64/268 essays
Processed 72/268 essays
Processed 80/268 essays
Processed 88/268 essays
Processed 96/268 essays
Processed 104/268 essays
Processed 112/268 essays
Processed 120/268 essays
Processed 128/268 essays
Processed 136/268 essays
Processed 144/268 essays
Processed 152/268 essays
Processed 160/268 essays
Processed 168/268 essays
Processed 176/268 essays
Processed 184/268 essays
Processed 192/268 essays
Processed 200/268 essays
Processed 208/268 essays
Processed 216/268 essays
Processed 224/268 essays
Processed 232/268 essays
Processed 240/268 essays
Processed 248/268 essays
Processed 256/268 essays
Processed 264/268 essays
Processed 268/268 essays
Test exact-match accuracy: 0.3843 (103/268)
GOLD 10 PRED 8 RAWOUT '8\nThe score assigned to the essay'
GOLD 5 PRED 8 RAWOUT '8\nThe score provided for the essay'
G

In [35]:
# CELL 16 — Save results
pd.DataFrame({"essay": essays, "gold": gold, "pred": preds, "raw_out": raw_outs}).to_csv("test_predictions.csv", index=False)
print("Saved test_predictions.csv. Adapter saved in ./qwen2.5-asap-p1-lora-adapter")


Saved test_predictions.csv. Adapter saved in ./qwen2.5-asap-p1-lora-adapter


In [36]:
!zip -r fine_tuned_model_try2.zip /kaggle/working

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/ (stored 0%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/checkpoint-78/ (stored 0%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/checkpoint-78/vocab.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 69%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/checkpoint-78/adapter_config.json (deflated 53%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/checkpoint-78/special_tokens_map.json (deflated 69%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/checkpoint-78/training_args.bin (deflated 52%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/checkpoint-78/scheduler.pt (deflated 56%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/checkpoint-78/README.md (deflated 66%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/checkpoint-78/merges.txt (deflated 57%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/checkpoint-78/rng_state.pth (deflated 25%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/checkpoint-78/adapter_model.safetensors (deflated 8%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/checkpoint-78/trainer_state.json (deflated 61%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/checkpoint-78/added_tokens.json (deflated 67%)
  adding: kaggle/working/qwen2.5-asap-p1-lora/checkpoint-78/op

In [37]:
from IPython.display import FileLink
FileLink(r'fine_tuned_model_try2.zip')


/kaggle/working/fine_tuned_model_try2.zip